# Exercise: Create Your Own Web Chatbot

This notebook demonstrates how to build a simple web Chatbot using `gradio` and `llama-cpp-python`.
It follows these steps:

1. **Install Dependencies**: Ensures all required packages are installed.
2. **Load Models**: Downloads an LLM model from Hugging Face.
3. **Create a Chatbot UI**: Use Gradio to define a Chatbot interface.
4. **Deploy the Chatbot**: Deploy the resulting web Chatbot and test it.

Run the final cell to see how the system responds to a test query.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required dependencies
!pip3 install gradio==5.16.0
!pip3 install huggingface_hub==0.28.1
!pip3 install llama-cpp-python==0.3.4 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124
!pip3 install langchain==0.3.17 langchain-core==0.3.33 langchain-community==0.3.14

In [ ]:
# Import necessary libraries
import gradio as gr
from huggingface_hub import hf_hub_download
from langchain_community.chat_models import ChatLlamaCpp

In [ ]:
# Download the LLM, you can search in Hugging Face
model_path = hf_hub_download(
    repo_id="Qwen/Qwen2.5-Coder-0.5B-Instruct-GGUF",
    filename="qwen2.5-coder-0.5b-instruct-q4_k_m.gguf",
    force_download=False,
)

In [ ]:
# Create the LLM
llm = ChatLlamaCpp(
    model_path=model_path,
    n_gpu_layers=25,
    stop=["<|im_end|>\n"],
    n_ctx=4096,
    max_tokens=4096,
    streaming=True,
    n_batch=256,
)

In [ ]:
# Create the llama handler
def llama_cpp(message, history):

    messages = history + [
        {
            "role": "user",
            "metadata": None,
            "content": message,
            "options": None,
        }
    ]
    response = ""

    for c in llm.stream(messages):
        response += c.content
        yield response

In [ ]:
# Create and launch the web chatbot
demo = gr.ChatInterface(
    llama_cpp,
    type="messages",
    flagging_mode="manual",
    flagging_options=[],
    save_history=True,
)

if __name__ == "__main__":
    demo.launch()
